In [ ]:
# %% [markdown]
# # Car Detection Feature Analysis
# 
# This notebook helps you understand the features used in car detection.

# %% [markdown]
# ## 1. Import Libraries

# %%
import sys
import os
sys.path.append('..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from src.utils.feature_extractor import CarFeatureExtractor
from src.utils.visualization import Visualizer
from src.models.detector import CarDetector

# %% [markdown]
# ## 2. Load Sample Image

# %%
# Load a sample car image
image_path = '../data/sample_car.jpg'  # Change to your image path
image = cv2.imread(image_path)
if image is not None:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.title('Sample Car Image')
    plt.axis('off')
    plt.show()
else:
    print("Please put a sample car image in data/sample_car.jpg")

# %% [markdown]
# ## 3. Extract and Visualize Features

# %%
# Initialize feature extractor
fe = CarFeatureExtractor()

# Visualize all features
fe.visualize_all_features(image)

# %% [markdown]
# ### 3.1 HOG Features
# Histogram of Oriented Gradients captures edge directions

# %%
features, hog_image = fe.extract_hog(image)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
ax1.set_title('Original')
ax1.axis('off')

ax2.imshow(hog_image, cmap='gray')
ax2.set_title(f'HOG Features (dim={len(features)})')
ax2.axis('off')
plt.show()

# %% [markdown]
# ### 3.2 LBP Features
# Local Binary Pattern captures texture information

# %%
hist, lbp_image = fe.extract_lbp(image)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
ax1.set_title('Original')
ax1.axis('off')

ax2.imshow(lbp_image, cmap='gray')
ax2.set_title('LBP Texture')
ax2.axis('off')

ax3.bar(range(len(hist)), hist)
ax3.set_title('LBP Histogram')
ax3.set_xlabel('Pattern')
ax3.set_ylabel('Frequency')
plt.show()

# %% [markdown]
# ## 4. Try Different Detection Methods

# %%
def detect_with_methods(image, methods=['canny', 'haar', 'hog']):
    """Try different detection methods."""
    
    results = {}
    
    # Method 1: Color-based segmentation
    if 'color' in methods:
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # Common car colors (you can adjust these ranges)
        color_ranges = [
            ((0, 50, 50), (10, 255, 255)),    # Red
            ((100, 50, 50), (130, 255, 255)), # Blue
            ((40, 50, 50), (80, 255, 255)),   # Green
            ((0, 0, 200), (180, 30, 255)),    # White
            ((0, 0, 0), (180, 255, 50))       # Black
        ]
        
        mask_total = np.zeros(image.shape[:2], dtype=np.uint8)
        for lower, upper in color_ranges:
            mask = cv2.inRange(hsv, lower, upper)
            mask_total = cv2.bitwise_or(mask_total, mask)
        
        results['color'] = mask_total
    
    # Method 2: Edge-based detection
    if 'canny' in methods:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 1.5)
        edges = cv2.Canny(blurred, 50, 150)
        results['canny'] = edges
    
    # Method 3: Contour-based
    if 'contour' in methods:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        contour_img = image.copy()
        cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
        results['contour'] = contour_img
    
    return results

# Test different methods
methods_results = detect_with_methods(image)

# Display results
fig, axes = plt.subplots(1, len(methods_results) + 1, figsize=(15, 4))

# Original
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')

# Results
for i, (method, result) in enumerate(methods_results.items(), 1):
    if method == 'contour':
        axes[i].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    else:
        axes[i].imshow(result, cmap='gray')
    axes[i].set_title(method.capitalize())
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 5. Interactive Feature Tuning
# Try different parameters and see the effect

# %%
from ipywidgets import interact, FloatSlider, IntSlider

@interact
def tune_canny(
    threshold1=IntSlider(min=0, max=300, step=10, value=50),
    threshold2=IntSlider(min=0, max=300, step=10, value=150),
    blur=IntSlider(min=1, max=15, step=2, value=5)
):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (blur, blur), 1.5)
    edges = cv2.Canny(blurred, threshold1, threshold2)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original')
    ax1.axis('off')
    
    ax2.imshow(edges, cmap='gray')
    ax2.set_title(f'Canny Edges (t1={threshold1}, t2={threshold2}, blur={blur})')
    ax2.axis('off')
    
    plt.show()

# %% [markdown]
# ## 6. Try the Real Detector

# %%
# Load the trained model (if available)
try:
    detector = CarDetector(model_type='yolov8n')
    detections = detector.detect(image, confidence=0.5)
    
    # Visualize detections
    vis = Visualizer({})
    vis.plot_image_with_bboxes(image, detections)
    
    print(f"Found {len(detections)} cars!")
    
except Exception as e:
    print(f"Model not available: {e}")
    print("Train the model first using: python scripts/train.py")